### Student:Rickard Sörlin

- I did not request a formal review for this lab, so there was no feedback to incoperate.

- AI tools like GitHub Copilot and ChatGPT were used as support to understand the existing code provided in notebooks like functions,arguments and methods
and to get a deeper explaination of concepts. They were also used whern i got stuck, needed to complete missing parts, or ecountered unexpected errors.

- Through the lab, I made sure I understood the suggestions i recieved before applying them. Using AI support helped improve my learning, since it encouraged me to ask follow-up questions such why a specific argument or methods was used and what alternatives could work. I then adapted the suggestions to fit my own solution.




# Lab 4: Parameter-efficient fine-tuning

Fine-tuning all parameters of pre-trained language models can be resource-intensive. Because of this, current research in natural language processing is looking into developing methods for adapting models to downstream tasks without full fine-tuning. These methods only tune a small number of model parameters while yielding performance comparable to that of a fully fine-tuned model.

In this lab, you will implement LoRA, one of the most well-known methods for parameter-efficient fine-tuning. LoRA stands for “Low-Rank Adaptation of Large Language Models” and was originally described in a research article by [Hu et al. (2021)](https://arxiv.org/abs/2106.09685).

Along the way, you will earn experience with [Hugging Face Transformers](https://huggingface.co/docs/transformers/en/index), a state-of-the-art library for training and deploying language models, as well as with several related libraries. In particular, you will learn a best-practice workflow for downloading a Transformer model and fine-tuning it on the downstream task of binary sentiment classification.

*Tasks you can choose for the oral exam are marked with the graduation cap 🎓 emoji.*

## Dataset

The data for this lab comes from the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). The full dataset consists of 50,000 highly polar movie reviews collected from the Internet Movie Database (IMDB). Here, we use a random sample consisting of 2,000 reviews for training and 500 reviews for evaluation.

To load the dataset, we use the [Hugging Face Datasets](https://huggingface.co/docs/datasets/en/index) library.

In [159]:
from datasets import load_dataset

imdb_dataset = load_dataset(
    "csv", data_files={"train": "train.csv", "eval": "eval.csv"}
)

imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 500
    })
})

As we can see, each sample in the dataset is a record with three fields: an internal index (`index`, an integer), the text of the review (`review`, a string), and the sentiment label (`label`, an integer – 1&nbsp;for “positive” and 0&nbsp;for “negative” sentiment).

Here is an example record:

In [160]:
imdb_dataset["train"][645]

{'index': 2981,
 'review': 'Brilliant execution in displaying once and for all, this time in the venue of politics, of how "good intentions do actually pave the road to hell". Excellent!',
 'label': 1}

## Tokeniser

As our pre-trained language model, we will use [DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert), a compact encoder model with 40% less parameters than BERT base. DistilBERT is not actually a *large* language model by modern standards and thus does not benefit as much from parameter-efficient fine-tuning as other models. However, it has the benefit of being light and fast, and can be run even on consumer hardware.

To feed the movie reviews to DistilBERT, we need to tokenise them and encode the resulting tokens as integers in the model vocabulary. We start by loading the DistilBERT tokeniser using the [Auto classes](https://huggingface.co/docs/transformers/en/model_doc/auto):

In [341]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

We then create a tokenised version of the dataset:

In [342]:
def tokenize_function(batch):
    return tokenizer(batch["review"], padding=True, truncation=True)


tokenized_imdb_dataset = imdb_dataset.map(tokenize_function, batched=True)

tokenized_imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

As we can see, tokenising adds two additional fields to each review: `input_ids` is the list of token ids corresponding to the review, and `attention_mask` is the list of indices specifying which tokens the encoder should attend to.

To avoid trouble when fine-tuning the model later, the next cell disables tokeniser parallelism.

In [343]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Trainer

In this section, we will set up our workflow for training and evaluating DistilBERT models. The central component in this workflow is the [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer), which provides extensive configuration options. Here, we leave most of these options at their default value. Two changes we *do* make are to enable evaluation of the trained model after each epoch, and to log the training and evaluation loss after every 5&nbsp;training steps (the default is 500).

In [344]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="tmp_trainer",
    eval_strategy="epoch",
    logging_steps=5,
)

In addition to the loss, we also track classification accuracy. For this we import the [Hugging Face Evaluate](https://huggingface.co/docs/evaluate/en/index) library and define a small helper function `compute_metrics()` that the trainer will call after each epoch.

In [165]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In the next cell we define a convenience function `make_trainer()` that creates a readily-configured trainer for a specified model (*model*). We will use this trainer both to train the model on the training section of the tokenised review dataset, and to evaluate it on the evaluation section.

In [345]:
from transformers import Trainer


def make_trainer(model):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_imdb_dataset["train"],
        eval_dataset=tokenized_imdb_dataset["eval"],
        compute_metrics=compute_metrics,
    )
    return trainer

## Full fine-tuning

In the rest of this notebook, we will work our way to the implementation of LoRA, and compare LoRA to traditional fine-tuning methods. Our first point of reference is a fully fine-tuned DistilBERT model.

We start by loading the pre-trained model:

In [346]:
from transformers import AutoModelForSequenceClassification

pretrained_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

pretrained_model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


The architecture of DistilBERT is that of a standard Transformer encoder with an embedding layer (`embeddings`) followed by a stack of six Transformer blocks (`transformer`) and a feedforward network with two linear layers (`pre_classifier` and `classifier`) and a final dropout layer (`dropout`).

### 🎈 Task 4.01: Counting the number of trainable parameters

One relevant measure in the context of parameter-efficient fine-tuning is the number of parameters that need to be changed when training a model. Your first task in this lab is to write a function `num_trainable_parameters()` that calculates this number for a given model.

In [347]:
def num_trainable_parameters(model):
    total_parameters = 0
    
    for param in model.parameters():
        if param.requires_grad:
            total_parameters += param.numel()
    return total_parameters


In [169]:
print("Number of trainable parameters:", num_trainable_parameters(pretrained_model))

Number of trainable parameters: 66955010


The function should implement the following specification:

> **num_trainable_parameters** (*model*)
>
> Returns the number of float-valued trainable parameters in the specified *model* as an integer.

#### 👍 Hint

The term *parameter* can refer to either complete tensors or the individual elements of these tensors. For example, a linear layer created by `nn.Linear(3, 5)` has 2&nbsp;tensor-valued parameters (a weight matrix and a bias vector) and 20&nbsp;float-valued parameters (the elements of these tensors). To get the tensor-valued parameters of a model, you can use the [`parameters()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters) method. A parameter is *trainable* if it requires gradient.

#### 🤞 Test your code

To test your code, apply your function to the pre-trained model. The correct number of float-valued trainable parameters for this model is 66,955,010.

### Fine-tuning

When we load the pre-trained model, the Hugging Face Transformers library warns us that the weights of the feedforward network have not yet been trained. To do so, in the next cell, we pass the pre-trained model to a trainer and initiate the fine-tuning process.

**⚠️ Please note that fine-tuning the model will take some time! ⚠️**

You can work on the other problems in this lab while you are waiting.

In [73]:
finetuned_trainer = make_trainer(pretrained_model)

finetuned_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.422600,0.325812,0.882000
2,0.020800,0.379034,0.906000
3,0.050300,0.447375,0.914000


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=750, training_loss=0.23099875203768414, metrics={'train_runtime': 1121.9055, 'train_samples_per_second': 5.348, 'train_steps_per_second': 0.669, 'total_flos': 794804391936000.0, 'train_loss': 0.23099875203768414, 'epoch': 3.0})

Because full fine-tuning is so resource-intensive, we save the fine-tuned model to disk:

In [ ]:
#finetuned_trainer.save_model("finetuned")

Later in this notebook, whenever you need the fully fine-tuned version of the model, you can load it as follows:

In [348]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained("finetuned")

### Convenience functions

Because we will repeat the steps we just took to fine-tune the pre-trained model several times in this notebook, we define two convenience functions:

In [321]:
def train(model):
    print("Number of trainable parameters:", num_trainable_parameters(model))
    trainer = make_trainer(model)
    trainer.train()
    return model

In [322]:
def evaluate(model):
    trainer = make_trainer(model)
    return trainer.evaluate()

## Tuning the final layers only

If full fine-tuning marks one end of the complexity spectrum, the other end is marked by only tuning the final layers of the transformer – the *head* of the model. In the case of DistilBERT, the head consists of the `pre_classifier` and `classifier` layers.

### 🎈 Task 4.02: Head-tuning

Implement the head-tuning strategy by coding the following function:

In [354]:
from copy import deepcopy
def make_headtuned_model():
    
    head_tuned_model = deepcopy(pretrained_model)
    for param in head_tuned_model.distilbert.parameters():
        param.requires_grad = False
    head_tuned_model.pre_classifier.weight.requires_grad = True
    head_tuned_model.classifier.weight.requires_grad = True
    trainer = make_trainer(head_tuned_model)
    trainer.train()
    return head_tuned_model


Here is the specification of this function:

> **make_headtuned_model** ()
>
> Returns a model that is identical to the pre-trained model, except that the head layers have been trained on the sentiment data. (The other parameters of the pre-trained model are left untouched.)

#### 👍 Hint

You freeze a parameter by setting its `requires_grad`-attribute to `False`.

Once you have an implementation of the head-tuning strategy, evaluate it on the evaluation data. How much accuracy do we lose when only training the final layers of the pre-trained model, compared to full fine-tuning?

In [355]:
headtuned_model = make_headtuned_model()

/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.569200,0.596263,0.720000
2,0.468600,0.536081,0.770000
3,0.520800,0.511159,0.808000


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


#### 🤞 Test your code

If you configured your model correctly, `num_trainable_parameters()` should show 592,130 trainable parameters.

In [356]:
print("Number of trainable parameters:", num_trainable_parameters(pretrained_model))
print("Number of trainable parameters:", num_trainable_parameters(finetuned_model))
print("Number of trainable parameters:", num_trainable_parameters(headtuned_model))

Number of trainable parameters: 66955010
Number of trainable parameters: 66955010
Number of trainable parameters: 592130


For future reference, we also save the head-tuned model:

In [ ]:
#make_trainer(headtuned_model).save_model("headtuned")


In [371]:
# evalutae models
print("Evaluating pretrained model:")
print(evaluate(pretrained_model))
print("Evaluating finetuned model:")
print(evaluate(finetuned_model))
print("Evaluating headtuned model:")
print(evaluate(headtuned_model))

Evaluating pretrained model:


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.6928027868270874, 'eval_model_preparation_time': 0.0009, 'eval_accuracy': 0.526, 'eval_runtime': 27.8617, 'eval_samples_per_second': 17.946, 'eval_steps_per_second': 2.261}
Evaluating finetuned model:


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.44737452268600464, 'eval_model_preparation_time': 0.0009, 'eval_accuracy': 0.914, 'eval_runtime': 19.6598, 'eval_samples_per_second': 25.433, 'eval_steps_per_second': 3.205}
Evaluating headtuned model:


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.5111589431762695, 'eval_model_preparation_time': 0.0014, 'eval_accuracy': 0.808, 'eval_runtime': 19.6876, 'eval_samples_per_second': 25.397, 'eval_steps_per_second': 3.2}


## Layer surgery

LoRA works by “wrapping” frozen layers from the pre-trained Transformer model inside adapter modules. Conventionally, this wrapping is only applied to the linear layers that transform the queries and values in the self-attention mechanism. To implement the wrapping, we need functions to extract and replace layers in a model. Your task in this section is to code these functions.

### 🎓 Task 4.03: Extracting layers

Code a function that extracts the query and value linear layers from a DistilBERT model:

In [358]:
pretrained_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [359]:
import torch.nn as nn
def extract(model):
    layers = {}
    for name, module in model.named_modules():
        if "attention.q_lin" in name or "attention.v_lin" in name:
            # print(name)
            layers[name] = model.get_submodule(name) 
    
    return layers


Implement this function to match the following specification:

> **extract** (*model*)
>
> Takes a DistilBERT model (*model*) and extracts the query and value linear layers from each block of the Transformer. Returns a dictionary mapping the DistilBERT module names of these layers to the layers themselves (instances of `nn.Linear`).

#### 👍 Hint

As we saw earlier, the DistilBERT model consists of a hierarchy of nested submodules. Each of these can be addressed by a fully-qualified string name. Use [`get_submodule()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.get_submodule) to retrieve a layer by name. You can hard-wire the names of the layers you want to extract.

#### 🤞 Test your code

To test your code, check the number of trainable float-valued parameters in the extracted layers. This number should be 7,087,104.

In [360]:

layers = extract(pretrained_model)
total = 0

for layer in layers.values():
    total += num_trainable_parameters(layer)
    
total

7087104

### 🎓 Task 4.04: Replacing layers

Next, code the inverse of the `extract()` function to replace selected layers of a module using a dictionary of named layers.

In [ ]:
import copy

def replace(model, named_layers):
    #Next, code the inverse of the `extract()` function to replace selected layers of a module using a dictionary of named layers.
    copy_model = copy.deepcopy(model) 
    for name, layer in named_layers.items(): 
        parent_name, attr_name = name.rsplit(".", 1)   
        parent = copy_model.get_submodule(parent_name)  # get the parent module
        setattr(parent, attr_name, layer) # set the new layer in the parent module
    return copy_model


Implement this function to match the following specification:

> **replace** (*model*, *named_layers*)
>
> Takes a DistilBERT model (*model*) and a dictionary in the format returned by `extract()` (*named_layers*) and injects the extracted layers into the model. More specifically, suppose that *named_layers* contains a key–value pair `(name, layer)`. Then the function replaces the submodule of *model* addressed by the fully-qualified string name `name` by the layer `layer`. Returns the modified model.

#### 👍 Hint

Use [`getattr()`](https://docs.python.org/3/library/functions.html#getattr) and [`setattr()`](https://docs.python.org/3/library/functions.html#setattr) to return or set the value of a named submodule.

#### 🤞 Test your code

To test your implementation, write code that (1)&nbsp;extracts the query and value linear layers from the fine-tuned model; (2)&nbsp;replaces these layers with clones with random weights; and (3)&nbsp;replaces these layers again with the original versions. Evaluating the modified model after step&nbsp;(2) should yield a near-random accuracy. Evaluating it again after step&nbsp;(3) should yield the original accuracy.

The following function should be helpful. It clones a linear layer, copying the weights and the bias from the original.

In [ ]:
import torch.nn as nn

def clone_linear(original):
    out_features, in_features = original.weight.shape # get the shape 
    copy = nn.Linear(in_features, out_features) # create new layer from shape 
    copy.load_state_dict(original.state_dict()) # load weights from original model to new layer
    return copy

In [363]:
trainer_fintuned_model = make_trainer(finetuned_model)
finetuned_result = trainer_fintuned_model.evaluate()
finetuned_result

/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.44737452268600464,
 'eval_model_preparation_time': 0.0011,
 'eval_accuracy': 0.914,
 'eval_runtime': 22.9921,
 'eval_samples_per_second': 21.747,
 'eval_steps_per_second': 2.74}

In [364]:

# change the weights a bit
orginal_layers = extract(finetuned_model)
modified_layers = {}
for name, layer in orginal_layers.items():
    device = layer.weight.device
    modified_layers[name] = clone_linear(layer).to(device)
    with torch.no_grad():
     
        modified_layers[name].weight += torch.randn_like(layer.weight)
        modified_layers[name].bias += torch.randn_like(layer.bias)

mixed_model = replace(finetuned_model, modified_layers)

trainer_mixed_model = make_trainer(mixed_model)
mixed_result = trainer_mixed_model.evaluate()
mixed_result



/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.7110881805419922,
 'eval_model_preparation_time': 0.0009,
 'eval_accuracy': 0.488,
 'eval_runtime': 21.4732,
 'eval_samples_per_second': 23.285,
 'eval_steps_per_second': 2.934}

In [365]:
backtothefuture_model = replace(mixed_model, orginal_layers)
trainer_backtothefuture_model = make_trainer(backtothefuture_model)
backtothefuture_result = trainer_backtothefuture_model.evaluate()
backtothefuture_result

/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.44737452268600464,
 'eval_model_preparation_time': 0.0012,
 'eval_accuracy': 0.914,
 'eval_runtime': 22.1982,
 'eval_samples_per_second': 22.524,
 'eval_steps_per_second': 2.838}

## Low-rank approximation

The basic idea behind LoRA is to conceptualise fine-tuned weights as a sum $W_0 + \Delta W$ of the weights from the pre-trained model, $W_0$, and a low-rank update matrix $\Delta W$. The goal of fine-tuning, then, is to learn the update matrix; this happens in the adapter layers.

Before we get to the implementation of the LoRA adapter layers, we first check to what extent the assumption that fine-tuning can be described by low-rank matrices holds true for DistilBERT. To do so, we will “cheat” and replace the query and value linear layers of the head-tuned model with low-rank approximations. The technical key to this is the truncated singular value decomposition (SVD).

### 🎓 Task 4.05: Low-rank matrix approximation

Your first task in this section is to implement the low-rank matrix approximation.

In [ ]:
import torch
def approximate(matrix, rank):
    # Perform SVD decomposition
    
    total_params = matrix.numel()
    #print(f"Total parameters in original matrix: {total_params}")

    U, Sigma, Vt = torch.linalg.svd(matrix, full_matrices=True) # 

    # Create diagonal matrix from Sigma and keep only top k singular values
    Sigma = torch.diag(Sigma)  # Convert to diagonal matrix
    Sigma_k = Sigma[:rank, :rank]  # Keep only top k x k submatrix

    # Keep only top k singular values/vectors
    U_k = U[:, :rank]
    Vt_k = Vt[:rank, :]
    
   # calculate parameters total
    total_params = U_k.numel() + Sigma_k.numel() + Vt_k.numel()
    #print(f"Total parameters in rank-{rank} approximation: {total_params}") 

    # Approximate the matrix
    matrix = torch.matmul(U_k, torch.matmul(Sigma_k, Vt_k))
    return matrix

Implement this function to match the following specification:

> **approximate** (*matrix*, *rank*)
>
> Takes a 2D-tensor (*matrix*) and an integer rank $r$ (*rank*), computes the truncated SVD with rank $r$ on the tensor, and returns the corresponding low-rank approximation matrix.

#### 👍 Hint

If you need a refresher on the low-rank matrix approximation, read the corresponding section from the Wikipedia article on the [Singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition#Low-rank_matrix_approximation). The truncated SVD is an extension of the full SVD; the latter can be computed using [`torch.linalg.svd()`](https://pytorch.org/docs/stable/generated/torch.linalg.svd.html).

#### 🤞 Test your code

To test your code, run the following cell. It creates a matrix `original` with rank $r \leq 8$ and after that the rank-$8$ approximation matrix `approximation`. You should find that the distance between the two matrices is very low.

In [373]:
original = torch.rand(768, 8) @ torch.rand(8, 384)
approximation = approximate(original, 8)
torch.dist(original, approximation)

Total parameters in original matrix: 294912
Total parameters in rank-8 approximation: 9280


tensor(0.0005)

### 🎓 Task 4.06: Approximated fine-tuned model (version 1)

In the next step, your task is to construct a version of the head-tuned model in which every query and value linear layer is replaced by a low-rank approximation of the corresponding layer from the fully fine-tuned model.

In [374]:
def make_approximated_model_1(rank):
    
    finetuned_layers = extract(finetuned_model)
    approximated_layers = {}
    
    for name, layer in finetuned_layers.items():
        
        layer = clone_linear(layer)
        weight_approximation = approximate(layer.weight, rank)
        
        with torch.no_grad():
            layer.weight.copy_(weight_approximation)
        
        approximated_layers[name] = layer
    
    approximated_model = replace(headtuned_model, approximated_layers)
    return approximated_model
    

Here is the specification of this function:

> **make_approximated_model_1** (*rank*)
>
> Takes an integer rank $r$ (*rank*) and returns a version of the head-tuned model in which every query and value linear layer is replaced by its $r$-approximated corresponding layer from the fully fine-tuned model.

In [387]:
print(f"Headtuned model accuracy:" , evaluate(headtuned_model)['eval_accuracy'])

/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Headtuned model accuracy: 0.808


Run the next cell to evaluate your model for different rank values. Start with the full rank and then halve the rank in each step. What is the lowest rank that still gives you a higher accuracy than the head-tuned model?

## The lowest rank that still gives higher accuracy is 384

In [386]:
rank = 768

for rank in [768, 384, 192, 96, 48,24,12,6,3,2,1]:
    
    approximated_model_1 = make_approximated_model_1(rank)
    print(f"rank-{rank} approximated model:", evaluate(approximated_model_1)['eval_accuracy']) 


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-768 approximated model: 0.868


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-384 approximated model: 0.838


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

rank-3 approximated model: 0.604


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-2 approximated model: 0.474


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-1 approximated model: 0.474


### 🎓 Task 4.07: Approximated fine-tuned model (version 2)

In the approximated model from the previous section, the truncated SVD is applied to the full weight matrix of the fine-tuned model: $W_0 + \Delta W$. In LoRA, the low-rank approximation only applies to the *update matrix* $\Delta W$, i.e., the difference between the fully fine-tuned weights and the pre-trained weights.

In [388]:
def make_approximated_model_2(rank):
    
    pretrained_layers = extract(pretrained_model)
    funetuned_layers = extract(finetuned_model)
    
    approximated_layers = {}
    
    for name, pretrained_layer in pretrained_layers.items():
        finetuned_layer = funetuned_layers[name]
        
        layer = clone_linear(pretrained_layer)
        
        # Compute the weight difference 
        deltaW = finetuned_layer.weight - pretrained_layer.weight
        # Approximate ONLY on DELTA W rank r approximation
        deltaW_approximation = approximate(deltaW, rank)
        
        with torch.no_grad():
            # Update the weights of the layer with pretrained weights + deltaW_approximation
            layer.weight.copy_(pretrained_layer.weight + deltaW_approximation)
        
        approximated_layers[name] = layer
    
    approximated_model = replace(headtuned_model, approximated_layers)
    return approximated_model

Implement the function to match the following specification:

> **make_approximated_model_2** (*rank*)
>
> Takes an integer rank $r$ (*rank*) and returns a version of the head-tuned model in which the weight matrix of every query and value linear layer is replaced by the sum $W_0 + \Delta W$, where $W_0$ is the weight matrix of the pre-trained model and $\Delta W$ is the rank-$r$ approximation of the update matrix, i.e., the difference between the fully fine-tuned weights and the pre-trained weights.

Run the next cell to evaluate your model for different rank values. Start with the rank from the approximated model from the previous section and then halve the rank in each step. What is the lowest rank that still gives you a higher accuracy than the head-tuned model?

## Rank 1 still gives higher accuracy ;O)

In [390]:
for rank in [768, 384, 192, 96, 48,24,12,6,3,2,1]:
    approximated_model_2 = make_approximated_model_2(rank)
    print(f"rank-{rank} approximated model:", evaluate(approximated_model_2)['eval_accuracy']) 

/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-768 approximated model: 0.87


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-384 approximated model: 0.87


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

rank-3 approximated model: 0.864


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-2 approximated model: 0.86


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


rank-1 approximated model: 0.856


In [391]:
approximated_model_2 = make_approximated_model_2(3)

evaluate(approximated_model_2)

/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.3819514214992523,
 'eval_model_preparation_time': 0.0009,
 'eval_accuracy': 0.864,
 'eval_runtime': 19.6647,
 'eval_samples_per_second': 25.426,
 'eval_steps_per_second': 3.204}

## Low-Rank Adaptation (LoRA)

In this section, you will implement the LoRA adapters and fine-tune the adapted model.

### 🎓 Task 4.08: Implement the adapter

A LoRA adapter implements the forward function

$$
y = x W_0 + x \Delta W = x W_0 + x A B
$$

where $W_0$ is a linear transformation from the pre-trained model and $\Delta W$ is a learned update matrix, deconstructed into the product $AB$ of two rank-$r$ matrices $A$ and $B$. LoRA scales the update matrix $\Delta W$ by a factor of $\alpha / r$, where $\alpha$ is a hyperparameter. (To keep the formula tidy, we ignore the fact that the linear transformation in the pre-trained model may additionally include a bias.)

In [ ]:
import torch.nn as nn


class LoRA(nn.Module):
    def __init__(self, pretrained, rank=12, alpha=24):
        super().__init__()
        self.pretrained = pretrained
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        self.A = nn.Linear(pretrained.in_features, rank, bias=False)
        self.B = nn.Linear(rank, pretrained.out_features, bias=False)
        
        with torch.no_grad():
            self.A.weight.normal_(mean=0.0, std=1.0)
            self.B.weight.zero_()
        
        for param in self.pretrained.parameters():
            param.requires_grad = False

    def forward(self, x):
        
        y0 = self.pretrained(x) 
        
        delta = self.B(self.A(x)) * self.scaling 
        return y0 + delta
        


Your code must comply with the following specification:

**__init__** (*self*, *pretrained*, *rank* = 12, *alpha* = 24)

> Initialises the LoRA adapter. This sets up the matrices $A$ and $B$ from the equation above. The matrix $A$ is initialised with random weights from a standard normal distribution; the matrix $B$ is initialised with zeros. The argument *pretrained* is the linear layer from the pre-trained model that should be adapted. The arguments *rank* and *alpha* are the rank $r$ and the hyperparameter $\alpha$ in the equation above.

**forward** (*self*, *x*)

> Sends an input *x* through the adapter, implementing the equation above.

### 🎓 Task 4.09: Inject the adapter into the pre-trained model

The final step is to construct an adapted model by injecting the LoRA adapters into the pre-trained model.

In [393]:
def make_lora_model(rank):
    model = copy.deepcopy(pretrained_model)
    layers = extract(model)
    
    for param in model.parameters():
        param.requires_grad = False
        
    for parameter in model.pre_classifier.parameters():
        parameter.requires_grad = True
    for parameter in model.classifier.parameters():
        parameter.requires_grad = True
        
    for name, layer in layers.items():
        lora_layer = LoRA(layer, rank=rank, alpha=rank*2)
        parent_name, attr_name = name.rsplit(".", 1)
        parent = model.get_submodule(parent_name)
        setattr(parent, attr_name, lora_layer)
    
    trainer = make_trainer(model)
    # time to train the LoR tuned model on the Imdb dataset for sentiment classification
    trainer.train()
    return model
 

Implement the function to match the following specification:

> **make_lora_model** (*rank*)
>
> Returns a model that is identical to the pre-trained model, except that the query and value linear layers have been wrapped in LoRA adapters, and the LoRA adapters and the head layers of the pre-trained model have been trained on the sentiment data. (The other parameters of the pre-trained model are left untouched.) The rank of the adapters is specified by the argument *rank*. The *alpha* value of the adapters is set to twice the rank (a common rule of thumb).

Run the next cell to evaluate your model for $r = 6$ and $\alpha = 12$. How many trainable parameters does the adapted model have? What accuracy do you get? How do these value relate to the number of trainable parameters and accuracy of the fully fine-tuned model, in terms of percentages?

### Finetuned model number of trainable parameters: 66955010
### Finetuned model accuracy:  0.914

### Lora model Rank-6 Number of trainable parameters:  702722
### Lora model Rank-6 accuracy:  0.886 
### Lora model Rank-6 Percentage compared to finetuned model:  1.049543566642735 %

In [400]:
print("Evaluating finetuned model accuracy: " ,(evaluate(finetuned_model)['eval_accuracy']))

/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluating finetuned model accuracy:  0.914


In [397]:
finetuned_model_param_count = num_trainable_parameters(finetuned_model)
print("Finetuned model number of trainable parameters:", finetuned_model_param_count)

for rank in [6]:
    lora_model = make_lora_model(rank)
    lora_model_param_count = num_trainable_parameters(lora_model)
    print("Lora model Rank-{rank} Number of trainable parameters: " ,lora_model_param_count)
    print("Lora model Rank-{rank} accuracy: " ,evaluate(lora_model)['eval_accuracy'])
    print ("Lora model Rank-{rank} Percentage compared to finetuned model: ", (lora_model_param_count/finetuned_model_param_count)*100 , "%")
    

Finetuned model number of trainable parameters: 66955010


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.580900,0.320425,0.882000
2,0.094300,0.313827,0.884000
3,0.196300,0.375383,0.886000


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Lora model Rank-{rank} Number of trainable parameters:  702722


/opt/anaconda3/envs/PyTorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Lora model Rank-{rank} accuracy:  0.886
Lora model Rank-{rank} Percentage compared to finetuned model:  1.049543566642735 %


## Alternatives to Transformer-based models

Even with methods for parameter-efficient fine-tuning, applying DistilBERT and other Transformer-based models comes at a significant cost – an investment that does not always pay off. In the final task of this lab, we ask you to explore a more traditional approach to classification and contrast it with the pre-training/fine-tuning approach of neural language models.

### 🎓 Task 4.10: Comparing with a non-neural classifier

Browse the web to find a tutorial on how to apply a classifier from the [scikit-learn](https://scikit-learn.org/stable/) library to the problem of sentiment classification and implement the method here in this notebook. We suggest you use [multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) or [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). (Once you have code for one method, it is easy to switch to the other.) Evaluate your chosen classifier on the IMDB dataset.

Questions to consider:

* Which classifier did you try? What results did you get? How long did it take you to train and run the classifier?
* What is your perspective on the trade-off between accuracy and resource requirements between the two approaches?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

In [445]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

train_texts = imdb_dataset["train"]["review"]
train_labels = imdb_dataset["train"]["label"]

eval_texts = imdb_dataset["eval"]["review"]
eval_labels = imdb_dataset["eval"]["label"]

#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer() 

X_train = vectorizer.fit_transform(train_texts)
X_eval  = vectorizer.transform(eval_texts)

model = LogisticRegression(max_iter=100)
#model = MultinomialNB()

model.fit(X_train, train_labels)
preds = model.predict(X_eval)

acc = accuracy_score(eval_labels, preds)
print(f"Accuracy: {acc:.4f}")



Accuracy: 0.8400




I tried both the LogisticRegression and the MultinomialNB (Naive Bayes) classifiers, as shown in the code above. The results from each step of the lab are listed below.
 
 ### Pre-trained DistilBERT (no fine-tuning) 
- Accuracy: 0.526
- Parameters: 66 955 010

### Fully fine-tuned DistilBERT
- Accuracy 0.914
- Parameters: 66 955 010

### Head-tuned DistilBERT
- Accuracy : 0.808
- Number of trainable parameters: 592 130

### LoRA-adapted DistilBERT
- Accuracy: 0.886
- Parameters: 702 722
- About 1.05 % of the parameters of fully fine-tuned DistilBert

The classifiers imported from the scikit-learn library were trained within about a second, which I found quite impressive, especially given that the accuracy was not far from the LoRA-adapted and fully fine-tuned DistilBERT models. At the same time, they require significantly fewer computational resources to train. This immediately made me think about many existing services in data centers, where i belive many B2B companies sometimes use large “sledgehammer” models for almost every problem or service they sell. In many of those cases, the energy consumption and hardware requirements could probably be much lower with a simpler model, if the team had the right knowledge and mindset. 

As an engineer at Schneider Electric, where our mission is to help people make the most of their energy and resources, I feel a bit concerned about where we are heading in terms of overconsumption of both compute and planetary resources. For me, this lab highlighted that we should not just use the most powerful model by default, but carefully choose techniques that are appropriate for the problem, the energy cost, and the real value they create not just for short-term gains, but for long-term sustainability as well.




**🥳 Congratulations on finishing this lab! 🥳**